# Inheco Incubator (Shaker)

<table style="width:100%; border-collapse:collapse;">
<tr>
<td style="width:60%; font-size:15px; line-height:1.7; vertical-align:top; padding-right:15px;">

<ul style="margin-top:0;">
  <li><a href="https://www.inheco.com/incubator-shaker.html" target="_blank"><b>OEM Link</b></a></li>
  <li><b>Communication Protocol / Hardware:</b> Serial (FTDI) / USB-A</li>
  <li><b>Communication Level:</b> Firmware (documentation shared by OEM)</li>
  <li>Same command set for:
    <ul>
      <li>Incubator “MP”</li>
      <li>Incubator “DWP”</li>
      <li>Incubator Shaker “MP”</li>
      <li>Incubator Shaker “DWP”</li>
    </ul>
  </li>
  <li><b>Incubator Shaker “MP”</b> VID:PID <code>0403:6001</code></li>
  <li>Takes in a single plate via a loading tray, heats it to the set temperature, and shakes it to the set RPM.</li>
</ul>

</td>

<td style="width:40%; text-align:center; vertical-align:middle;">
  <img src="img/inheco_incubator_shaker_mp_dwp.png" width="500"/><br>
  <i>Figure: Inheco Incubator Shaker MP & DWP models</i>
</td>
</tr>
</table>

## About the Machine(s)

Inheco incubator shakers are modular machines used for plate storage, temperature control and shaking.
They differentiate themselves:
- **heater shakers** ... heat a material on which a plate is being placed; open-access; non-uniform temperature distribution around the plate; enables shaking of plate.
- **incubator shakers** ... an enclosed chamber that is being heated and houses a plate; plate access is controlled via a loading tray and a door; *highly uniform temperature distribution around the plate*; enables shaking of plate.

The Inheco incubator devices come in 4 versions, dependent on (1) whether they provide a shaking feature & (2) the size of plates they accept:


| **RTS Code** | **Shaking Feature** | **Plate Format** | **Device Identifier** | **Typical Model** |
|:-------------:|:--------------:|:----------------:|:----------------------|:------------------|
| `0` | ❌ No | MP (Microplate) | `incubator_mp` | INHECO Incubator MP | 
| `1` | ✅ Yes | MP (Microplate) | `incubator_shaker_mp` | INHECO Incubator Shaker MP | 
| `2` | ❌ No | DWP (Deepwell Plate) | `incubator_dwp` | INHECO Incubator DWP | 
| `3` | ✅ Yes | DWP (Deepwell Plate) | `incubator_shaker_dwp` | INHECO Incubator Shaker DWP | 


```{note}
Note: All 4 machines can be controlled with the same PyLabRobot Backend, called `InhecoIncubatorShakerBackend`!
```

---
## Setup Instructions (Physical)

<!-- ![quadrants](img/inheco_incubator_shaker_physical_setup_overview.png) -->
<table style="width:100%; border-collapse:collapse; margin-top:10px;">
<tr>
<td style="text-align:center; vertical-align:middle;">
  <img src="img/inheco_incubator_shaker_physical_setup_overview.png" width="950" style="border-radius:8px;"/>
  <br>
  <i>Figure: Physical setup overview of the Inheco Incubator Shaker system</i>
</td>
</tr>
</table>


To facilitate integration, multiple devices can be placed on top of each other to form an Incubator Shaker Stack (see infographic above).
Up to 6 machines can be placed into the same stack. 
When using more than 6 machines, you must build multiple stacks (none can contain more than 6 machines).
The machines in a single stack can be of any of the 4 types.

The benefit of this setup is that only **one** power cable and only **one** USB cable have to be plugged into the machine at the very bottom of a machine (i.e. stack index 0).
Machines above the bottom one only need to be connected with the machine below it using the 15-pin SUB-D connectors that come with each machine when bought from Inheco.

```{note}
Note: In PyLabRobot, each machine is controlled via its own instance of the `InhecoIncubatorShakerBackend`.
```

<table>
<tr>
<td style="font-size:15px; line-height:1.6; width:60%; vertical-align:top; padding-right:10px;">

To connect an <code>InhecoIncubatorShakerBackend</code> there are two identifiers that uniquely characterise every physical machine:<br><br>

<ol style="margin-left: 20px;">
<li><b>DIP switch identifier</b> — located on the back of the bottom machine; it defines the DIP switch configuration for the entire stack above it.<br>
(<i>Note: You must set this DIP switch manually; see instructions below.</i>)</li>
<li><b>Stack index number</b> — the position a machine occupies within its stack.<br>
(<i>Note: This cannot be reassigned after connecting the stack unless the physical stack arrangement is changed.</i>)</li>
</ol>

<h3>Setting the DIP switch to generate a machine address</h3>

The DIP switch at the back of each machine consists of 4 pins that can be set into an <code>UP</code> / <code>0</code> or a <code>DOWN</code> / <code>1</code> position.

(<i>Note:</b> There are two more pins to the left of the DIP switch pins. They are not involved in setting the DIP switch address, and should be left in their <code>DOWN</code> position.</i>)</li>

This represents <i>binary encoding</i>:
<ul>
<li>All pins at <code>0</code> → DIP switch is set to address <code>0</code></li>
<li>All pins at <code>1</code> → DIP switch is set to address <code>15</code> (2<sup>4</sup>-1)</li>
</ul>

</td>

<td style="width:40%; text-align:center; vertical-align:middle;">
<img src="img/inheco_incubator_shaker_dip_switch_addressing.png" width="500"/><br>
<i>Figure: DIP switch layout to generate different identifiers/addresses</i>
</td>
</tr>

</table>

---
## Setup Instructions (Programmatic)

After the two cables have been connected to the bottom-most Inheco Incubator Shaker, you only have to instantiate the `InhecoIncubatorShakerBackend` and give it the correct `dip_switch_id` & `stack_index`.

```{note}
Before a connection has been established the incubator shaker's front LED blinks.
After the connection has succesfully been made, the LED will continuously be on.
```

In [1]:
import logging
from pylabrobot.io import LOG_LEVEL_IO
from datetime import datetime

current_date = datetime.today().strftime('%Y-%m-%d')
protocol_mode = "execution"

# Create the shared file handler once
fh = logging.FileHandler(f"{current_date}_testing_{protocol_mode}.log", mode="a")
fh.setLevel(LOG_LEVEL_IO)
formatter = logging.Formatter(
    "%(asctime)s [%(levelname)s] %(name)s - %(message)s"
)
fh.setFormatter(formatter)

# Configure the main pylabrobot logger
logger_plr = logging.getLogger("pylabrobot")
logger_plr.setLevel(LOG_LEVEL_IO)
if not any(isinstance(h, logging.FileHandler) and h.baseFilename == fh.baseFilename
           for h in logger_plr.handlers):
    logger_plr.addHandler(fh)

# Other loggers can reuse the same file handler
logger_manager = logging.getLogger("manager")
logger_device = logging.getLogger("device")

for logger in [logger_manager, logger_device]:
    logger.setLevel(logging.DEBUG)  # or logging.INFO
    if not any(isinstance(h, logging.FileHandler) and h.baseFilename == fh.baseFilename
               for h in logger.handlers):
        logger.addHandler(fh)

# START LOGGING
logger_manager.info("START AUTOMATED PROTOCOL")


In [2]:
import time
import asyncio


In [3]:
from pylabrobot.storage.inheco import IncubatorShakerStack, InhecoIncubatorShakerBackend


In [4]:
iis_stack_backend = InhecoIncubatorShakerBackend(dip_switch_id = 2)

iis_stack = IncubatorShakerStack(backend=iis_stack_backend)

await iis_stack.setup(verbose=True)

Connected to Inheco Incubator Shaker Stack on /dev/cu.usbserial-130
DIP switch ID of bottom unit: 2
Number of connected units: 2
Unit composition: {0: 'incubator_shaker_mp', 1: 'incubator_shaker_dwp'}


In [5]:
iis_stack.size_x, iis_stack.size_y, iis_stack.size_z

(149.0, 268.5, 227.5)

In [6]:
iis_stack.loading_tray_status

{0: 'closed', 1: 'closed'}

In [7]:
await iis_stack[0].open()

iis_stack.loading_tray_status

{0: 'open', 1: 'closed'}

In [8]:
iis_50 = iis_stack[1]

In [9]:
await iis_50.close()


In [15]:
await iis_stack.open_all()

iis_stack.loading_tray_status

{0: 'open', 1: 'open'}

In [16]:
await iis_stack.close_all()

iis_stack.loading_tray_status

{0: 'closed', 1: 'closed'}

In [12]:
await iis_stack.start_all_temperature_control(target_temperature=37)

await asyncio.sleep(15)

await iis_stack.get_all_temperatures()


{0: 30.1, 1: 34.3}

In [13]:
await iis_stack.stop_all_temperature_control()


In [14]:
iis_stack.children

[ResourceHolder(name='unit-0-loading-tray', location=Coordinate(030.500, -150.500, 051.200), size_x=127.76, size_y=85.48, size_z=0, category=resource_holder),
 ResourceHolder(name='unit-1-loading-tray', location=Coordinate(030.500, -150.500, 139.700), size_x=127.76, size_y=85.48, size_z=0, category=resource_holder)]

In [10]:
await iis_stack[0].close()


In [16]:
await iis_stack_backend.request_incubator_type(stack_index=1)

'incubator_shaker_dwp'

In [15]:
await iis_stack_backend.close(stack_index=1)

In [20]:
print(getattr(iis_stack, "_size_z", None))
print(getattr(iis_stack, "size", None))
58.0
None

58.0
None


In [ ]:
await iis_stack.send_command("RTS", stack_index=0)

In [ ]:
iis_0 = InhecoIncubatorShakerUnit(backend=iis_stack, index=0)

In [10]:
await iis_stack_backend.close(stack_index=0)

In [5]:
await iis_stack.send_command("RTS", stack_index=1)

'3'

In [6]:
await iis_stack.request_number_of_connected_machines(stack_index=0)

2

In [7]:
iis_stack.unit_composition

{0: 'incubator_shaker_mp', 1: 'incubator_shaker_dwp'}

In [8]:
# Time how long the machine takes to reach target temperature 
# using standard Python - no need to re-invent the wheel

await iis_stack.start_temperature_control(37, stack_index=0)

# start_time = time.time()


In [11]:
iis_stack.temperature_control_status

{0: False, 1: False}

In [14]:
iis_stack.shaking_status

{0: False, 1: False}

In [13]:
# Figure-eight shaking example

await iis_stack.shake(
    pattern="orbital",
    rpm=500,
    stack_index=0
)

await asyncio.sleep(5)

await iis_stack.stop_shaking(stack_index=0)


In [2]:
stack_0_interface = InhecoIncubatorShakerStack(
    dip_switch_id = 2
)

await stack_0_interface.setup()


In [3]:
await stack_0_interface.send_command("RTS")

'1'

In [4]:
iis_0 = incubator_shaker_0 = InhecoIncubatorShakerBackend(
    dip_switch_id = 2,
    stack_index = 0
)

await iis_0.setup(verbose=True)


Connected to INHECO incubator_shaker_mp on /dev/cu.usbserial-130
Machine serial number: 2013
Firmware version: IncShak_C_V3.50_04/2012


```{note}
If you are interested in seeing information about the machine you are connecting to, you can set the `.setup()` optional argument `verbose` to `True`:
1. type of Inheco Incubator
2. the port used
3. the machine's serial number
4. the firmware version the machine runs on
```

## Usage: 1 Stack - 1 Machine

### Sensing Plate Presence

Inheco incubator shakers contain an internal, reflection-based plate sensor.
This is very useful e.g. when someone has forgotten their plate in the incubator 👀

In [6]:
await iis_0.request_plate_in_incubator()


False

### Using Loading Tray

In [7]:
await iis_0.open()


In [8]:
await iis_0.close()


```{warning}
**On parallelization of commands to machines in the same incubator shaker stack**

Each machine in the same stack communicates via the same USB(-A to -B) cable.
As a result, you cannot use parallelization (e.g. `aysyncio.gather()`) to communicate with machines in the same stack.

This means you cannot open all incubator shakers in the same stack at the same time.

However, if you arrange your Inheco Incubators into different stacks this should still be possible.
```

### Temperature Control

In [7]:
# Show current temperature

current_temp = await iis_0.get_temperature()

current_temp


21.0

In [8]:
# Time how long the machine takes to reach target temperature 
# using standard Python - no need to re-invent the wheel

await iis_0.start_temperature_control(37)

start_time = time.time()


In [9]:
# Quick check of how the temperature increases for 5 sec

for x in range(5):
    
    current_temp = await iis_0.get_temperature(sensor="main")
    print(current_temp)

    time.sleep(1)


21.1
21.6
22.5
23.5
24.4


<table style="width:100%; border-collapse:collapse;">
  <tr>
    <td style="width:60%; font-size:15px; line-height:1.7; vertical-align:top; padding-right:20px;">
      <p style="margin-top:0;">The Inheco Incubator (Shaker) contains three independent temperature sensors:</p>
      <ol style="margin-top:0; margin-bottom:10px; padding-left:20px;">
        <li>main sensor — close to the door/front, inside the machine</li>
        <li>validation sensor — back, inside the machine</li>
        <li>boost sensor — on heating foil, inside the machine</li>
      </ol>
      <p>
        By default, <code>iis_0.get_temperature()</code>’s argument is set to
        <code>sensor="main"</code>.  
        This can be changed to any of the following:
      </p>
      <ul style="margin-top:0; padding-left:20px;">
        <li><code>"main"</code></li>
        <li><code>"dif"</code></li>
        <li><code>"boost"</code></li>
        <li><code>"mean"</code> — takes all three sensors’ measurements and returns their geometric mean</li>
      </ul>
    </td>
    <td style="width:40%; text-align:center; vertical-align:middle;">
      <img src="img/inheco_incubator_shaker_t_sensor_positioning.png"
           alt="Inheco Incubator Shaker sensor positions"
           style="width:300px; border-radius:6px; margin-bottom:8px;"/>
      <br>
      <i style="font-size:13px; color:#6c757d;">
        Figure: Inheco Incubator Shaker Temperature Sensor Positioning
      </i>
    </td>
  </tr>
</table>


In [10]:
# Wait until target temperature has been reach

temp_reached = await iis_0.wait_for_temperature(sensor = "mean", show_progress_bar = True)

elapsed_time = time.time() - start_time

print(f"{temp_reached} ℃, {round(elapsed_time, 1)} sec")


Waiting for target temperature 37.00 °C...

[███████████████████████████████████████-] 36.87 °C (Δ=0.13 °C, target=37.00 °C))
✅ Target temperature reached.
36.87 ℃, 35.7 sec


In [11]:
# Simple stopping of temperature control without stopping the machine itself

await iis_0.stop_temperature_control()


### Shaking Control

Only Incubator "Shakers" can use shaking commands.

During `.setup()` the machine will check whether it is an `incubator_shaker` ("MP" or "DWP") and the Python backend only allows shaking commands being sent to the machine if it is an `incubator_shaker`.

In [6]:
# Simplest usage

await iis_0.shake(rpm=800)

await asyncio.sleep(5)

await iis_0.stop_shaking()


Inheco incubator shakers support precise, programmable motion in both the **X** and **Y** axes.
The resulting shaking pattern is defined by five parameters:

- **Amplitude in X** (`Aₓ`, 0–3 mm)
- **Amplitude in Y** (`Aᵧ`, 0–3 mm)
- **Frequency in X** (`fₓ`, 6.6–30.0 Hz)
- **Frequency in Y** (`fᵧ`, 6.6–30.0 Hz)
- **Phase shift** (`φ`, the angular offset between X and Y motion, in degrees)

Different combinations of these parameters produce circular, linear, elliptical, or
figure-eight movement paths.

---

#### Predefined Shaking Patterns in PyLabRobot

To simplify configuration, PyLabRobot provides predefined motion presets that map common use cases to specific parameter combinations:

| Pattern | Description | Parameter relationship | Required speed attribute |
|----------|--------------|------------------------|---------------------------|
| `orbital` | Circular shaking | `Aₓ = Aᵧ`, `φ = 90°`, `fₓ = fᵧ` | `rpm` |
| `elliptical` | Elliptical motion | `Aₓ ≠ Aᵧ`, `φ = 90°`, `fₓ = fᵧ` | `rpm` |
| `figure_eight` | Figure-eight (Lissajous) motion | `Aₓ ≈ Aᵧ`, `φ = 90°`, `fᵧ = 2 fₓ` | `rpm` |
| `linear_x` | Linear motion along X | `Aᵧ = 0` | `frequency_hz` |
| `linear_y` | Linear motion along Y | `Aₓ = 0` | `frequency_hz` |

```{note}
The default behaviour of `.shake()` uses...
- an orbital shaking pattern,
- x amplitude = 3 mm,
- y amplitude = 3 mm.

(see “Simplest usage” example above)


In [15]:
# Orbital shaking example with modified amplitudes

await iis_0.shake(
    pattern="orbital",
    rpm=800,
    amplitude_x_mm=2.0,
    amplitude_y_mm=2.0
)

await asyncio.sleep(5)

await iis_0.stop_shaking()


In [16]:
# Elliptical shaking example with modified amplitudes

await iis_0.shake(
    pattern="elliptical",
    rpm=800,
    amplitude_x_mm=2.5,
    amplitude_y_mm=2.5
)

await asyncio.sleep(5)

await iis_0.stop_shaking()


In [11]:
# Figure-eight shaking example

await iis_stack.shake(
    pattern="figure_eight",
    rpm=400,
    stack_index=1
)

await asyncio.sleep(5)

await iis_0.stop_shaking()


InhecoError: ASEND failed with error E03: Invalid operand

If you feel adventorous, see the math that goes into the calculation of different shaking patterns here:

<details>
<summary><b>📘 How PyLabRobot Implements Inheco Shaking Patterns (Mathematical Overview)</b></summary>

INHECO incubator shakers move a plate by oscillating the platform in two directions — **X** and **Y** — at programmable amplitudes, frequencies, and phase offsets.

---

**The Core Equations**

The motion of the platform is described by two sinusoidal functions:

\[
\begin{aligned}
x(t) &= Aₓ \sin(2\pi fₓ t) \\
y(t) &= Aᵧ \sin(2\pi fᵧ t + φ)
\end{aligned}
\]

Where:

| Symbol | Meaning | Example |
|:--|:--|:--|
| `Aₓ`, `Aᵧ` | Amplitudes (mm) — how far the plate moves in X and Y | 2.5 mm |
| `fₓ`, `fᵧ` | Frequencies (Hz) — how fast each axis oscillates | 10 Hz, 20 Hz |
| `φ` | Phase shift (°) — timing offset between X and Y | 0°, 90°, 180° |

Each axis moves smoothly back and forth like a spring.  
When these two motions combine, they trace elegant paths such as circles, ellipses, or figure-eights.

---

**Pattern Intuition**

Different shaking patterns are created by adjusting the relationships between these parameters:

| Pattern | Conditions | Description |
|:--|:--|:--|
| **Linear X** | `Aᵧ = 0` | Motion only along X (back-and-forth line) |
| **Linear Y** | `Aₓ = 0` | Motion only along Y |
| **Orbital** | `Aₓ = Aᵧ`, `fₓ = fᵧ`, `φ = 90°` | Perfect circular motion |
| **Elliptical** | `Aₓ ≠ Aᵧ`, `fₓ = fᵧ`, `φ = 90°` | Elongated circle (ellipse) |
| **Figure-Eight (Lissajous)** | `Aₓ ≈ Aᵧ`, `fᵧ = 2 fₓ`, `φ = 90°` | Double-loop path shaped like ∞ |

---

**Example: Figure-Eight Motion**

In firmware terms:

SSP20,20,100,200,90
ASE1


corresponds to:

- `Aₓ = Aᵧ = 2.0 mm`
- `fₓ = 10.0 Hz`
- `fᵧ = 20.0 Hz`
- `φ = 90°`

This combination makes the platform’s Y motion twice as fast as its X motion —  
the resulting path is a **Lissajous figure**, visually resembling a “figure-8”.

---

**Why This Matters**

By controlling these parameters precisely:
- The **mixing efficiency** can be tuned to the liquid’s viscosity.
- The **path geometry** affects shear stress and aeration.
- **Repeatable motion profiles** ensure reproducibility across runs.

Understanding this relationship helps you select the right pattern
(`orbital`, `elliptical`, `figure_eight`, etc.) for your experiment.

</details>



---
## Usage: 1 Stack - Multiple Machine

If more than one machine is used in an Inheco Incubator Shaker "stack", all will use the same serial port.

Machines are then distinguished via the `stack_index` attribute given to `InhecoIncubatorShakerBackend` during instantiation:

In [10]:
iis_1 = inheco_incubator_shaker_1 = InhecoIncubatorShakerBackend(
    dip_switch_id = 2,
    stack_index = 1
)

await iis_1.setup(verbose=True)


Connected to INHECO incubator_shaker_dwp on /dev/cu.usbserial-130
Machine serial number: 977
Firmware version: IncShak_C_V3.50_04/2012


In [11]:
await iis_1.open()


In [14]:
await iis_1.close()


In [21]:
# Parallelize shaking of different incubators with different shaking conditions
# did someone say "Design of Experiments" 👀📊

await iis_0.start_temperature_control(37)
await iis_1.start_temperature_control(29)

await iis_0.wait_for_temperature(sensor = "mean", show_progress_bar = True)
await iis_1.wait_for_temperature(sensor = "mean", show_progress_bar = True)


await iis_0.shake(
    pattern="orbital",
    rpm=500,
)

await iis_1.shake(
    pattern="figure_eight",
    rpm=800,
)

await asyncio.sleep(10)

await iis_0.stop_temperature_control()
await iis_1.stop_temperature_control()

await iis_0.stop_shaking()
await iis_1.stop_shaking()


Waiting for target temperature 37.00 °C...

[███████████████████████████████████████-] 36.87 °C (Δ=0.13 °C, target=37.00 °C)
✅ Target temperature reached.
Waiting for target temperature 29.00 °C...

[█████████████---------------------------] 29.20 °C (Δ=0.20 °C, target=29.00 °C)
✅ Target temperature reached.


```{note}
If you develop a small script that you find yourself re-using and that goes beyond the simple "hello world, inheco incubator shaker"-style examples here, please consider contributing it back to the PyLabRobot community as a Cookbook Recipe.
```

---
## Usage: Multiple Stack - Multiple Machine

To connect another stack with its own machines, simply instantiate the new `InhecoIncubatorShakerBackend`s using the correct DIP switch identifier/address.

(set on the back of the bottom-most machine):


In [22]:
# commented out because original implementation had only 2 machines at their disposal

# iis_2 = inheco_incubator_shaker_2 = InhecoIncubatorShakerBackend(
#     dip_switch_id = 3,
#     stack_index = 0
# )

# await iis_2.setup(verbose=True)

# iis_3 = inheco_incubator_shaker_3 = InhecoIncubatorShakerBackend(
#     dip_switch_id = 3,
#     stack_index = 1
# )

# await iis_3.setup(verbose=True)


### Closing Connection

Standard PyLabRobot way of closing the communication connection.

In [23]:
await iis_0.stop(); await iis_1.stop()
